In [1]:
import pandas as pd
from utils import run_time_series
import pandapower as pp

In [2]:
def create_network():
    # Create empty network
    net = pp.create_empty_network()

    # Create buses
    buses = {}
    for i in range(18):
        if i == 0:
            buses[i] = pp.create_bus(net, vn_kv=20.0, name=f"Bus {i}")
        else:
            buses[i] = pp.create_bus(net, vn_kv=0.4, name=f"Bus {i}")

    # External grid connection at Bus 0
    pp.create_ext_grid(net, bus=buses[0], vm_pu=1.02, name="Grid Connection")

    # Transformer between Bus 0 and Bus 1
    pp.create_transformer(net, hv_bus=buses[0], lv_bus=buses[1], std_type="0.4 MVA 20/0.4 kV", name="Transformer")

    # Create lines
    lines = [
        (1, 2), (2, 3), (3, 4), (4, 5), (5, 6), (6, 7), (7, 8),
        (1, 9), (9, 10), (10, 11), (11, 12), (12, 13), (13, 14), (14, 15), (15, 16), (16, 17)
    ]
    for line in lines:
        pp.create_line(net, from_bus=buses[line[0]], to_bus=buses[line[1]], length_km=0.1, std_type="NAYY 4x50 SE", name=f"Line {line[0]}-{line[1]}")

    # Add loads and generators (adjusting the indices to start from Bus 1)
    for i in range(1, 18):
        pp.create_load(net, bus=buses[i], p_mw=0.01, q_mvar=0.005, name=f"Load {i}")
        pp.create_sgen(net, bus=buses[i], p_mw=0.01, q_mvar=0.005, name=f"SGen {i}")
    return net

In [3]:
your_gen_data = pd.read_csv("GenerationData_B.csv", index_col=0)
your_load_data = pd.read_csv("LoadData_B.csv", index_col=0)
your_net = create_network()
#Descending 
your_gen_order = [13, 12, 14, 11, 15, 9, 10, 16, 2, 3, 4, 8, 0, 5, 7, 1, 6]
#your_load_order = [13, 6, 4, 2, 0, 14, 15, 8, 7, 5, 16, 10, 12, 11, 1, 3, 9]
#Normal
your_gen_order = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
#your_load_order = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
#Reverse 
#your_gen_order = your_gen_order[::-1]
#your_load_order = your_load_order[::-1]

In [4]:
# save your_gen and your_load in single data frame and save to csv
df_save = pd.DataFrame({'generation_order':your_gen_order,'load_order':your_load_order})

In [5]:
Group_Letter = 'E' # change this to your group letter
df_save.to_csv(f'Group_{Group_Letter}_Assignment_2_Data_Set_C_order.csv',index=False)

In [6]:
# load again to check if it is correct
df = pd.read_csv(f'Group_{Group_Letter}_Assignment_2_Data_Set_C_order.csv')

In [7]:
your_gen_order_loaded = df['generation_order'].values
your_load_order_loaded = df['load_order'].values

In [8]:
your_gen_order == your_gen_order_loaded
your_load_order == your_load_order_loaded

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True])

In [9]:
res_ext, res_lines = run_time_series(your_gen_data, your_load_data, your_net, your_gen_order, your_load_order, results_suffix="evaluation")

No time steps to calculate are specified. I'll check the datasource of the first controller for avaiable time steps
100%|███████████████████████████████████████████████████████████████████████████████████████| 24/24 [00:03<00:00,  7.06it/s]


In [10]:
all(res_lines.max(axis=1) <= 100.0) # has to be True

True

In [11]:
res_lines.max(axis=1).sum() # to be minimized

116.44563792678511

In [12]:
pp.runpp(your_net)
your_net.res_ext_grid

,p_mw,q_mvar
0,0.004862,-0.000014
